# Notebook 1a: Creates a 'sample' file

## Objectives

- Create a 'quick & dirty' sample file from earlier step 0 - to see if the API extract processes worked as it should have.
- To use for EDA in the next step 1b

## Input(s)

- The business Variables (below) as gleaned from the business requirements email.
- This includes list of ETF tickers, the data date range, and time interval

## Output(s)
- generates a 'raw_merged.csv' file and stores it into the resources\raw_merged folder

## Section 1: Import Python Libraries

In [1]:
# Usual Dataframe Library
import pandas as pd

# No System Warnings
import warnings
warnings.filterwarnings('ignore')

# Check the current working directory
import os
print(os.getcwd()) 

c:\Users\tishs\vscode-projects\etf-close-price\jupyter_notebooks


# Section 2: Define Business Variables

In [ ]:
# Once we find out everything is working, we can use the full list (below) to see if we can run multiple ETF models successfully
ticker_list2 = [
    'SKY', 'VOO', 'IVV', 'VTI', 'ITOT',    # Core S&P/Total Market
    'QQQ', 'QQQE',                         # Nasdaq/Growth
    'IWM', 'IWF', 'IWD', 'MDY',            # Small/Mid/Growth/Value
    'XLK', 'XLF', 'XLE', 'XLV',             # Tech/Finance/Energy
    'TLT', 'BND', 'HYG',                   # Bonds
    'VEU', 'EFA'                           # International (US-listed)
]

#needed for both API data extractions
start_date = "2020-01-01"
end_date = "2025-12-31"

#Only required for Yahoo Finance data extraction
interval = "1d"

# Section 3: Define Functions to
- Load ALL the ETF files using the above ticker list 2, using a for loop to stack them on top of each other
- Load the FRED macroeconomic data
- Merge the two dataframes
- Save this file, ready to be used in the next step (EDA)

In [ ]:
def load_and_stack_raw_tickerdf(tickers, interval):
     
    if isinstance(tickers, str): #adds single ticker to a new list (just in case a string is passed instead of a list)
        tickers = [tickers]

    data = {} #empty dictionary to hold dataframes for all tickers

    for ticker in tickers: #gets the list of tickers and goes through them one by one
        
        #variable to find the correct file path to read in the raw data files
        filename_in = f"..\\resources\\raw_files\yfinance\{ticker}_{interval}_data.csv"
        
        #reads in the raw data file for each ticker and adds it to the dictionary
        data[ticker] = pd.read_csv(filename_in, 
                                   parse_dates=['Date'],            #converts the 'Date' column to Pandas datetime format
                                   index_col='Date')                #sets the 'Date' column as the index of the dataframe
        
        #prints the shape of each loaded dataframe - I find this useful to see if the ticker data loaded correctly
        print(f"Loaded {ticker}: {data[ticker].shape}")

    dfs = [] #from a dictionary to an empty list of dataframes for concatenation

    for ticker, df in data.items():

        df_reset = df.reset_index()                                 #resets index to have 'Date' as a column again
        df_reset['Ticker'] = ticker                                 #adds a new column with the ticker name as we will need this to identify which ticker each row belongs to after concatenation
        dfs.append(df_reset)                                        #adds the modified dataframe to the list above

    #concatenates all dataframes in the list into a single dataframe and returns it    
    return pd.concat(dfs, axis=0, ignore_index=True) 

In [ ]:
def stack_and_merge_raw_etf_econ_data(ticker, interval):

    raw_stacked_df = load_and_stack_raw_tickerdf(ticker, interval) #uses the function above to load and stack the raw data files into a single dataframe
    
    #usual print statement to check the shape of the stacked dataframe for audit purposes
    print(f"\nFinal stacked raw dataframe shape: {raw_stacked_df.shape}") 
    
    macro_df = pd.read_csv(f"..\\resources\\raw_files\\fred\\fred_macro_data.csv") #loads in the single macroeconomic data file created in notebook 0

    #for audit purposes - prints the shape of the macroeconomic data file 
    print('\nMacro file shape:', 
          macro_df.shape)

    #need to 'align' the date formats in both dataframes before merging them together
    raw_stacked_df["Date"] = pd.to_datetime(raw_stacked_df["Date"], 
                                            format='mixed',            # so as to handle any varied & inconsistent date formats
                                            dayfirst=True)             # cuts out the guessing factor, by asking pandas to see ambiguous dates as day first (US vs UK date format styles)
    
    macro_df["Date"] = pd.to_datetime(macro_df["Date"], 
                                      format='mixed',                   # see abpve
                                      dayfirst=True)                    # see above

    merged1 = raw_stacked_df.merge(macro_df, 
                                   on="Date",                           # merges both dataframes on the Date column - as required by the business case
                                   how="inner").fillna(0)               # only select and keep dates from both dataframes - then fill any blanks after the joining with 0

    #Some admin is needed here - Moving the Ticker column to the second position - at the front of the file after Date
    col = merged1.pop("Ticker")
    merged1.insert(1, 
                   "Ticker", 
                   col)

    print('\nMerged file shape:',                                       #seeing what the shape of the new file looks like
          merged1.shape)

    merged1.to_csv(f"..\\resources\\raw_merged\\raw_merged.csv",        #finally creating a new file and storing it in the raw_merged folder
                   index=False)

# Section 4 - Running the functions and creating the file for EDA

In [30]:
stack_and_merge_raw_etf_econ_data(ticker_list2, interval)

Loaded SKY: (1507, 7)
Loaded VOO: (1507, 8)
Loaded IVV: (1507, 8)
Loaded VTI: (1507, 8)
Loaded ITOT: (1507, 8)
Loaded QQQ: (1507, 8)
Loaded QQQE: (1507, 8)
Loaded IWM: (1507, 8)
Loaded IWF: (1507, 8)
Loaded IWD: (1507, 8)
Loaded MDY: (1507, 8)
Loaded XLK: (1507, 8)
Loaded XLF: (1507, 8)
Loaded XLE: (1507, 8)
Loaded XLV: (1507, 8)
Loaded TLT: (1507, 8)
Loaded BND: (1507, 8)
Loaded HYG: (1507, 8)
Loaded VEU: (1507, 8)
Loaded EFA: (1507, 8)

Final stacked raw dataframe shape: (30140, 10)

Macro file shape: (2190, 47)

Merged file shape: (30120, 56)


### Note for the Bug List

- Not an actual bug as such, but those with a keen eye will noticed the disparity between the stacked ETF raw dataframe shape and merged file shapes above
- I've recorded this on an Excel file in the same directory (for sake of audit/completeness again)
- This was because the ETF file had data for a date that the FRED Macro file did not - 20 records were therefore dropped for the date - 02/01/2020
- I found a similar date disparity issue later on - as sometimes a working day in one file is a holiday date in another!